In [10]:
# this code is written by Changyi Yang used for DS Discovery program

import torch.nn as nn
import torch

import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn 
from sklearn.model_selection import GridSearchCV

import pandas as pd

In [11]:
torch. __version__

'1.13.0'

In [12]:
# do the data preprocessing

from sklearn.preprocessing import StandardScaler

def preprocessing(file_name):
    data = pd.read_csv(file_name, skiprows=[0]) # skip the first line
    data = data.iloc[:, 1:] # skip the first row

    ss = StandardScaler()


    data = ss.fit_transform(data)

    return data, ss



In [13]:
# define the dataset classes

from torch.utils.data import Dataset, DataLoader

class ReactorData(Dataset):
    def __init__(self,data, sequence_length, start_percent = 0, end_percent = 1):
        
        
        length = data.shape[0]
        data = data[ int(length * start_percent)  : int(length * end_percent)]
        
        # print(data.shape)
        
        
        self.labels = data[:, -1:]
        self.data = data[:, 0:-1]
        

        
        self.sequence_length = sequence_length
        
    def __len__(self):
        return len(self.labels)//self.sequence_length
    
    def __getitem__(self,idx):
        idx = idx * self.sequence_length
        
        return (torch.tensor(self.data[idx : idx+ self.sequence_length])).double(), \
    (torch.tensor(self.labels[idx : idx+ self.sequence_length])).double()

        

In [14]:
# load the data
data, ss = preprocessing('binnedpebbles.csv')

# load the data
sequence_length = 10

training_data = ReactorData(data, sequence_length= sequence_length, start_percent= 0, end_percent= 0.75)
testing_data = ReactorData(data, sequence_length= sequence_length, start_percent= 0.75, end_percent= 1)





In [15]:
# print(training_data[0][1])
# print(training_data[0][0].shape)
# print(training_data[0][1].shape)

# print(len(training_data[0]))
# print(len(training_data))

In [16]:
# define the neural nets
class SimpleLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, lstm_nums_layer, dropout):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        
        self.LSTM = nn.LSTM(input_dim, hidden_dim, lstm_nums_layer, batch_first = True)
        self.dropout = nn.Dropout(dropout)
        
        self.hidden_to_output = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, input):
        # print(input.shape)
        
        hidden_state, _ = self.LSTM(input)
        
        # print(hidden_state.shape)
        output = self.dropout(hidden_state)
        output = self.hidden_to_output(output)
        
        
        return output
    
    


In [17]:
# train the model

input_dim = training_data[0][0].shape[1]
output_dim = training_data[0][1].shape[1]


# print(output_dim)

# some adjustable hyper-parameters

hidden_dim = 64
num_hidden_layers = 2
batch_size = 32
learning_rate = 1e-3
weight_decay = 1e-5
epoch_num = 128
dropout = 0.2


def train(hidden_dim, num_hidden_layers, batch_size, learning_rate, weight_decay, epoch_num, dropout):
    model = SimpleLSTM(input_dim, hidden_dim, output_dim, num_hidden_layers, dropout)
    model = model.double()

    train_dataloader = DataLoader(training_data, batch_size = batch_size)


    # the chosn loss function and optimizer
    loss_fn = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay= weight_decay)

    for epoch in range(epoch_num):
    
        for batch, (X, y) in enumerate(train_dataloader):
            model.zero_grad()
        
            pred = model(X)
        
            # print(X.shape)
            # print(y.shape)
        
            loss = loss_fn(pred, y)
        
            # backpropagation
        
            loss.backward()
            optimizer.step()
        
            loss = loss.item()
        
        #if epoch % 5 == 0:
         #   print("The loss is {} in epoch {}".format(loss ,epoch))
    return model
            

#print(f"The training is ended, the final loss is {loss}.")
print("Bye")

Bye


In [18]:
def test(model, testing_data):
    X_test = (torch.tensor(testing_data.data)).double()
    y_test = (torch.tensor(testing_data.labels)).double()

    y_pred = model(X_test)
    loss = nn.MSELoss()
    print(f"The test loss is {loss(y_pred, y_test).item()}")
    return loss(y_pred, y_test).item()

In [19]:
trained_model = train(hidden_dim, num_hidden_layers, batch_size, learning_rate, weight_decay, epoch_num, dropout)
test(trained_model, testing_data)

The test loss is 0.03379000274163063


0.03379000274163063

In [26]:
best_loss = 1
best_params = [0] * 5

for i in range(5, 8):
    hidden_dim = 2**i
    for k in range(-1, 2):
        dropout = (2**k)*0.1
        for l in range(2, 6):
            learning_rate = 10**(-l)
            for m in range(2, 6):
                weight_decay = 10**(-l)
                for n in range(2, 6):
                    batch_size = 2**n
                    trained_model = train(hidden_dim, 2, batch_size, learning_rate, weight_decay, 100, dropout)
                    loss = test(trained_model, testing_data)
                    if loss < best_loss:
                        best_model = trained_model
                        best_loss = loss
                        best_params[0] = hidden_dim
                        best_params[1] = dropout
                        best_params[2] = learning_rate
                        best_params[3] = weight_decay
                        best_params[4] = batch_size


torch.save(best_model.state_dict(),'simple_best.pth')                       
print("iterations completed")                        
print(best_params)
print(best_loss)                       

The test loss is 0.05199353278384666
The test loss is 0.0419791265380754
The test loss is 0.035434186889278546
The test loss is 0.021591525851350726
The test loss is 0.028525334587811233
The test loss is 0.02094623542162918
The test loss is 0.025475065061123684
The test loss is 0.025091435477892263
The test loss is 0.04281886208003941
The test loss is 0.02116086442419024
The test loss is 0.019408211159897427
The test loss is 0.029037997453465043
The test loss is 0.040875772633668456
The test loss is 0.02165992417703914
The test loss is 0.026408493277056737
The test loss is 0.022049433751107288
The test loss is 0.03482279330283606
The test loss is 0.02611996039947959
The test loss is 0.03656699765206594
The test loss is 0.026284137260116
The test loss is 0.02555383873286713
The test loss is 0.026174909483185793
The test loss is 0.03350624450468945
The test loss is 0.025921855027407896
The test loss is 0.020523220371840154
The test loss is 0.02797619402543317
The test loss is 0.023926900

In [20]:
# input_dim, hidden_dim, output_dim, lstm_nums_layer, dropout
best_model = SimpleLSTM(input_dim, 128,1, 2, 0.05)

best_model.load_state_dict(torch.load('simple_best.pth'))
best_model.to(torch.double)
best_model.eval()
print(test(best_model, testing_data))

The test loss is 0.014216222676088824
0.014216222676088824


In [22]:
import plotly.graph_objects as go
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch import nn

# Assuming 'best_model', 'data', and 'ss' (StandardScaler) are defined

X = (torch.tensor(data[:, :-1])).double()
y = (torch.tensor(data[:, -1])).double()

y_pred = best_model(X)

y_pred = y_pred.detach().numpy()

X_pred = np.append(data[:, :-1], y_pred.reshape(-1, 1), axis=1)

y_original = ss.inverse_transform(data)[:, -1]

y_pred_original = ss.inverse_transform(X_pred)[:, -1]

loss = nn.MSELoss()
print(loss(best_model(X), y).item())

# Plotly graph
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(len(y_original))),
                         y=y_original,
                         mode='lines',
                         name='Real'))

fig.add_trace(go.Scatter(x=list(range(len(y_pred_original))),
                         y=y_pred_original,
                         mode='lines',
                         name='Predict'))

fig.update_layout(title='The real and predict Keff with model trained on stable state',
                  xaxis_title='Index',
                  yaxis_title='Value')

fig.show()

import plotly.offline as pyo
pyo.plot(fig, filename='stable_one.html')


2.0545045927092933


/Users/changyiyang/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning:

Using a target size (torch.Size([234])) that is different to the input size (torch.Size([234, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



'stable_one.html'

In [52]:
import plotly.graph_objects as go
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch import nn

# Assuming 'best_model', 'data', and 'ss' (StandardScaler) are defined

X = (torch.tensor(data[:, :-1])).double()
y = (torch.tensor(data[:, -1])).double()

y_pred = best_model(X)

y_pred = y_pred.detach().numpy()

X_pred = np.append(data[:, :-1], y_pred.reshape(-1, 1), axis=1)

y_original = ss.inverse_transform(data)[:, -1]

y_pred_original = ss.inverse_transform(X_pred)[:, -1]

loss = nn.MSELoss()
print(loss(best_model(X), y).item())

# Plotly graph
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(len(y_original))),
                         y=y_original,
                         mode='lines',
                         name='Real'))

fig.add_trace(go.Scatter(x=list(range(len(y_pred_original))),
                         y=y_pred_original,
                         mode='lines',
                         name='Predict'))

# Add a vertical line at x = 150
fig.add_shape(type='line',
              x0=150, x1=150,
              y0=min(np.min(y_original), np.min(y_pred_original)),
              y1=max(np.max(y_original), np.max(y_pred_original)),
              yref='y',
              xref='x',
              line=dict(color='rgba(255, 0, 0, 0.5)', width=2),
              name='Train Test Split')

fig.update_layout(title='The Real and Predict Keff with Model Trained on Stable State',
                  xaxis_title='Index',
                  yaxis_title='Keff',
                  annotations=[dict(x=150,
                                   y=max(np.max(y_original), np.max(y_pred_original)),
                                   xref="x", yref="y",
                                   text="Train-Test Split",
                                   showarrow=True,
                                   arrowhead=2,
                                   ax=0,
                                   ay=-30)])

# Add a vertical line at x = 150
fig.add_shape(type='line',
              x0=150, x1=150,
              y0=min(np.min(y_original), np.min(y_pred_original)),
              y1=max(np.max(y_original), np.max(y_pred_original)),
              yref='y',
              xref='x',
              line=dict(color='rgba(0, 0, 0, 1)', width=2),  # Change color to black
              name='Train Test Split')


fig.show()


2.0545045927092933


/Users/changyiyang/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning:

Using a target size (torch.Size([234])) that is different to the input size (torch.Size([234, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

